<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/Notebooks2023/Argilla_configure_dataset-v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created: 31 Dec 2023 (Happy New Year!!)

Aaron, Kennedy, Israel, Vishnu.

Modification History:

4 Jan 2024:
Aaron, Frank, Othniel, Vishnu: Changed the data schema to a simpler format. Bye-bye "for_supervised_fine_tuning" format.

Description: This notebook configures the dataset.

Pre-requisites:
the following notebooks are already run:

1. Create the raw dataset in HF hub.

2. Manually Create a HF Spaces deployment of Argilla.

This notebook will Configure the argilla dataset

//And later you may need to

//add records in the argilla dataset from the raw dataset

//annotate the dataset in UI

## Install Libraries

Install the latest version of Argilla in Colab, along with other libraries and models used in this notebook.

In [ ]:
!pip install argilla

Prerequisites

Deploy Argilla Server on [HF Spaces](https://huggingface.co/new-space?template=argilla/argilla-template-space).


More info on Installation [here](../getting_started/installation/deployments/deployments.html).

## Secretes needed




* `ARGILLA_API_URL`: It is the url of the Argilla Server.
  * If you're using HF Spaces, it is constructed as `https://[your-owner-name]-[your_space_name].hf.space`.
* `ARGILLA_API_KEY`: It is the API key of the Argilla Server. It is `owner` by default.
* `HF_TOKEN`: It is the Hugging Face API token. It is only needed if you're using a [private HF Space](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html#deploy-argilla-on-spaces). You can configure it in your profile: [Setting > Access Tokens](https://huggingface.co/settings/tokens).
* `workspace`: admin


In [2]:
import argilla as rg
from argilla._constants import DEFAULT_API_KEY

In [3]:
from google.colab import userdata
api_url= userdata.get('my_argilla_url')
api_key= userdata.get('my_argilla_key')

import argilla as rg
rg.init(api_url=api_url, api_key=api_key)

# # If you want to use your private HF Space
# rg.init(extra_headers={"Authorization": f"Bearer {hf_token}"})

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:168: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


### Use a Custom Template


In [4]:
#Create a custom dataset configuration
custom_dataset = rg.FeedbackDataset(
    fields = [
        rg.TextField(name="background"),
        rg.TextField(name="prompt"),
        rg.TextField(name="response", title="Final Response"),
    ],
    questions=[
        rg.TextQuestion(
            name="response_correction",
            description="If you think the response is not accurate, please, correct it.",
            required=True,
        ),
    ],
    guidelines="Please, read the question carefully and try to answer it as accurately as possible."
)
custom_dataset


FeedbackDataset(
   fields=[TextField(name='background', title='Background', required=True, type='text', use_markdown=False), TextField(name='prompt', title='Prompt', required=True, type='text', use_markdown=False), TextField(name='response', title='Final Response', required=True, type='text', use_markdown=False)]
   questions=[TextQuestion(name='response_correction', title='Response_correction', description='If you think the response is not accurate, please, correct it.', required=True, type='text', use_markdown=False)]
   guidelines=Please, read the question carefully and try to answer it as accurately as possible.)
   metadata_properties=[])
)

In [5]:
try:
    remote_dataset= custom_dataset.push_to_argilla(name="fgan-annotate-dataset", workspace="admin")
except:
    pass

INFO:argilla.client.feedback.dataset.local.mixins:✓ Dataset succesfully pushed to Argilla
INFO:argilla.client.feedback.dataset.local.mixins:RemoteFeedbackDataset(
   id=9d63097e-8e18-4341-b446-64f59a8e2759
   name=fgan-annotate-dataset
   workspace=Workspace(id=6196e1fe-7cc5-4ef4-b608-d98a8bc8fbc8, name=admin, inserted_at=2024-01-02 14:17:38.856061, updated_at=2024-01-02 14:17:38.856061)
   url=https://vishnuramov-itu-t-build-a-thon.hf.space/dataset/9d63097e-8e18-4341-b446-64f59a8e2759/annotation-mode
   fields=[RemoteTextField(id=UUID('1afd1754-1dc6-48a8-951f-b7f37ce8e270'), client=None, name='background', title='Background', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('0de91bf8-000a-4393-9f40-0aa775fb89a8'), client=None, name='prompt', title='Prompt', required=True, type='text', use_markdown=False), RemoteTextField(id=UUID('f8e4f450-f39d-4761-a899-18fe65dabfd8'), client=None, name='response', title='Final Response', required=True, type='text', use_markdow

-------------

